# Web Scraper for 19 Cities

This scraper extracts and organizes data into three main DataFrames:
1. **`all_projects_df`**: Contains all projects from the websites.
   - Columns: `Project URL`, `Project Title`, `Project Description`, `Proposal Count`, `City`

2. **`all_proposals_df`**: Contains all proposals under projects.
   - Columns: `URL`, `Title`, `Proposed for Project`, `Description`, `Author`, `Comments`, `Supporters`, `City`

3. **`all_comments_df`**: Contains all comments under projects and proposals.
   - Columns: `URL`, `Project`, `Text`, `Author`, `Likes`, `Dislikes`, `Date`, `City`


In [115]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Updated function to extract proposals from a project page
def extract_proposals(soup, base_url):
    proposals = []
    proposal_items = soup.find_all('div', class_='resource-item proposal-list-item')

    for proposal in proposal_items:
        # Extract title
        title_tag = proposal.find('a', class_='resource-item--title')
        title = title_tag.get_text(strip=True) if title_tag else None

        # Extract URL
        url = base_url + title_tag['href'] if title_tag and 'href' in title_tag.attrs else None

        # Extract description
        description_tag = proposal.find('div', class_='resource-item--description')
        description = description_tag.get_text(strip=True) if description_tag else None

        # Extract author/username
        author_tag = proposal.find('a', class_='resource-item--author')
        author = author_tag.get_text(strip=True) if author_tag else None

        # Extract number of comments
        comments_tag = proposal.find('span', class_='comments')
        comments = int(comments_tag.get_text(strip=True).split()[0]) if comments_tag else 0

        # Extract number of supporters
        supporters_tag = proposal.find('span', class_='total-supports')
        supporters = int(supporters_tag.get_text(strip=True).split()[0]) if supporters_tag else 0

        # Extract parent project
        project_tag = proposal.find('a', class_='breadcrumbs-item')
        proposed_for_project = project_tag.get_text(strip=True) if project_tag else None

        proposals.append({
            'URL': url,
            'Title': title,
            'Proposed for Project': proposed_for_project,
            'Description': description,
            'Author': author,
            'Comments': comments,
            'Supporters': supporters,
        })
    return proposals


# Function to extract city name from the base URL
def extract_city_name(base_url):
    # Words to remove from the city name
    remove_words = ['mitmachen', 'Mitmachen', 'mitwirken', 'Smarte', 'region', 'unser', 'mitgestalten', 'gestalten', 'machmit', 'dialog', 'consul', 'www', 'de', 'https', 'com']

    # Split the URL into parts (by '.' or '/')
    parts = base_url.replace('https://', '').replace('http://', '').split('.')
    all_parts = [part.split('/')[0] for part in parts]  # Handle cases where "/" exists after domain

    # Remove known unwanted words and empty strings
    filtered_parts = [part for part in all_parts if part.lower() not in remove_words and part]

    # Return the first relevant part (assumes city name is left after filtering)
    city = filtered_parts[0].replace('-', ' ').capitalize() if filtered_parts else "Unknown"

    # Remove unwanted words from city name
    for word in remove_words:
        city = city.replace(word, '')

    return city.strip().capitalize()


# Update project scraping to exclude comments
def scrape_project_page_with_proposals(url, base_url, city):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to load project page: {url}")
        return None, []

    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract project title
    title_tag = soup.find('title')
    project_title = title_tag.get_text(strip=True) if title_tag else None

    # Extract project description
    content_div = soup.find('div', class_='flex-layout')
    description = content_div.get_text(strip=True) if content_div else None

    # Extract proposals
    proposals = extract_proposals(soup, base_url=base_url)

    return {
        'Project URL': url,
        'Project Title': project_title,
        'Project Description': description,
        'Proposal Count': len(proposals),
    }, proposals


# Modified function to scrape projects with proposals
def scrape_projects_with_proposals(main_url, base_url):
    response = requests.get(main_url)
    if response.status_code != 200:
        print(f"Failed to load main projects page: {main_url}")
        return pd.DataFrame(), pd.DataFrame()

    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all project links
    links = soup.find_all('a', class_='resource-item--title')
    project_links = [base_url + link['href'] for link in links if 'href' in link.attrs]

    projects = []
    all_proposals = []

    for project_url in project_links:
        try:
            project_data, proposals = scrape_project_page_with_proposals(
                project_url, base_url, extract_city_name(base_url)
            )
            if project_data:
                projects.append(project_data)
                all_proposals.extend(proposals)
        except Exception as e:
            print(f"Error scraping project at {project_url}: {e}")

    return pd.DataFrame(projects), pd.DataFrame(all_proposals)

# List of websites (fixed flensburg-mitmachen.de base_url)
websites = [
    {"main_url": "https://wuerzburg-mitmachen.de/projekts", "base_url": "https://wuerzburg-mitmachen.de"},
    {"main_url": "https://mitmachen.siegburg.de/projekts", "base_url": "https://mitmachen.siegburg.de"}, 
    {"main_url": "https://mitmachen.jena.de/projekts", "base_url": "https://mitmachen.jena.de"},
    {"main_url": "https://mitmachgemeinde.de/projekts", "base_url": "https://mitmachgemeinde.de"},
    {"main_url": "https://bamberg-gestalten.de/projekts", "base_url": "https://bamberg-gestalten.de"},
    {"main_url": "https://mitmachen-pforzheim.de/projekts", "base_url": "https://mitmachen-pforzheim.de"},
    {"main_url": "https://bochum-mitgestalten.de/projekts", "base_url": "https://bochum-mitgestalten.de"},
    {"main_url": "https://unser.muenchen.de/projekts", "base_url": "https://unser.muenchen.de"},
    {"main_url": "https://mitreden.ilzerland.bayern/projekts", "base_url": "https://mitreden.ilzerland.bayern"},
    {"main_url": "https://stutensee-mitwirken.de/projekts", "base_url": "https://stutensee-mitwirken.de"},
    {"main_url": "https://consul.unterschleissheim.de/projekts", "base_url": "https://consul.unterschleissheim.de"},
    {"main_url": "https://machmit.kempten.de/projekts", "base_url": "https://machmit.kempten.de"},
    {"main_url": "https://consul.detmold-mitgestalten.de/projekts", "base_url": "https://consul.detmold-mitgestalten.de"},
    {"main_url": "https://flensburg-mitmachen.de/projekts", "base_url": "https://flensburg-mitmachen.de"},  # Fixed URL
    {"main_url": "https://mitmachen.amberg.de/projekts", "base_url": "https://mitmachen.amberg.de"},
    {"main_url": "https://mitmachen.smarte-region-linz.de/projekts", "base_url": "https://mitmachen.smarte-region-linz.de"},
    {"main_url": "https://mitgestalten.trier.de/projekts", "base_url": "https://mitgestalten.trier.de"},
    {"main_url": "https://machmit.augsburg.de/projekts", "base_url": "https://machmit.augsburg.de"}
]


# Initialize empty DataFrames for all projects and proposals
all_projects_df = pd.DataFrame()
all_proposals_df = pd.DataFrame()

# Main loop to scrape all websites
for site in websites:
    main_url = site["main_url"]
    base_url = site["base_url"]

    city = extract_city_name(base_url)

    try:
        # Scrape projects and proposals
        projects_df, proposals_df = scrape_projects_with_proposals(main_url, base_url)

        # Add a 'City' column to all DataFrames
        projects_df['City'] = city
        proposals_df['City'] = city

        # Append results to the combined DataFrames
        all_projects_df = pd.concat([all_projects_df, projects_df], ignore_index=True)
        all_proposals_df = pd.concat([all_proposals_df, proposals_df], ignore_index=True)
    except Exception as e:
        print(f"Failed to scrape {main_url} - {e}")


Error scraping project at https://flensburg-mitmachen.dehttps://survey.lamapoll.de/Publikumspreis-Kommune-bewegt-Welt-2024: HTTPSConnectionPool(host='flensburg-mitmachen.dehttps', port=443): Max retries exceeded with url: /survey.lamapoll.de/Publikumspreis-Kommune-bewegt-Welt-2024 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000018D29CE5AB0>: Failed to resolve 'flensburg-mitmachen.dehttps' ([Errno 11001] getaddrinfo failed)"))


In [131]:
all_proposals_df.head()

,URL,Title,Proposed for Project,Description,Author,Comments,Supporters,City
0,https://wuerzburg-mitmachen.de/proposals/110-autofreier-bischofshut,Autofreier Bischofshut,Zukunftskonzepte für die Innenstadt,"Wir fordern die Ausrufung des Klimanotstands, damit Belange unseres Klimas vor das wirtschaftlic...",Letzte Generation Würzburg,0.0,20.0,Wuerzburg
1,https://wuerzburg-mitmachen.de/proposals/109-e-scooter-verbieten,E Scooter verbieten,Zukunftskonzepte für die Innenstadt,E Scooter sollten (im Innenstadtbereich) verboten werden. Diese werden häufig willkürlich abgest...,Ccmuet,0.0,2.0,Wuerzburg
2,https://wuerzburg-mitmachen.de/proposals/108-barrierefrei-ins-nautiland-lgs,Barrierefrei ins Nautiland/LGS,Zukunftskonzepte für die Innenstadt,Nautiland - neu.\r\nUmweltstation - neu.\r\nZellertorauffahrt - neu.\r\nLeider fehlen die barrie...,AASeuffert,0.0,2.0,Wuerzburg
3,https://wuerzburg-mitmachen.de/proposals/107-kinderabenteuer-indoor-spielplatz-smaland,Kinderabenteuer / Indoor Spielplatz / Smaland,Zukunftskonzepte für die Innenstadt,Es gibt zwar schon den FunPark für Kinder mit Trampolinhalle etc. in der Nähe der Nürnberger Str...,ABlitz,0.0,0.0,Wuerzburg
4,https://wuerzburg-mitmachen.de/proposals/106-banke-und-grun-im-neu-gestalteten-bereich-karmelite...,"Bänke und ""Grün"" im neu gestalteten Bereich Karmelitenstraße/Vierröhr...",Zukunftskonzepte für die Innenstadt,Die Baustelle von der Karmelitenstraße zum Vierröhrenbrunnen wurde vor kurzem abgeschlossen. Die...,Ccmuet,0.0,12.0,Wuerzburg


In [132]:
all_projects_df.head()

,Project URL,Project Title,Project Description,Proposal Count,City
0,https://wuerzburg-mitmachen.de/grombuehl-zukunftssicher,Energetisches Quartierskonzept für Grombühl,"Grombühl 2040 - ein SzenarioDie Straßen Grombühls sind grüner, ruhiger und voller Leben. Das „Qu...",0,Wuerzburg
1,https://wuerzburg-mitmachen.de/mobilitaetsplan,Mobilitätsplan 2040,Mobilitätsplan 2040 für die Stadt Würzburg: Jetzt mitmachen!Aktuell erstellt die Stadt Würzburg ...,0,Wuerzburg
2,https://wuerzburg-mitmachen.de/zukunftsregion,Zukunftsregion Würzburg,Zukunftsregion Würzburg: Jetzt aktiv mitgestalten!Die Stadt und der Landkreis Würzburg wollen ih...,0,Wuerzburg
3,https://wuerzburg-mitmachen.de/zukunftskonzepte-fuer-die-innenstadt,Zukunftskonzepte für die Innenstadt,"Wie soll die Würzburger Innenstadt von morgen aussehen? Was wünschen sich Bürger:innen, Einzelhä...",24,Wuerzburg
4,https://wuerzburg-mitmachen.de/klimaanpassung,Klimaanpassung,Klimaanpassungsstrategie für die Stadt Würzburg: Jetzt mitmachen!Würzburg - Seit Anfang 2024 era...,14,Wuerzburg


#### BurgerBudgets in Jena (2024, 23, 22)

In [126]:
# # URLs for the budgets
# budget_urls = {
#     2024: "https://mitmachen.jena.de/buergerbudget",
#     2023: "https://mitmachen.jena.de/buergerbudget-2023",
#     2022: "https://mitmachen.jena.de/buergerbudget-2022"
# }

# # Updated function to scrape and clean a budget table for a given year
# def scrape_and_clean_budget_table(url, year):
#     response = requests.get(url)
#     if response.status_code != 200:
#         print(f"Failed to load URL: {url}")
#         return None
    
#     soup = BeautifulSoup(response.content, 'html.parser')
#     table = soup.find('table', id='budget-investments-compatible')  # Locate the table by its ID
    
#     if not table:
#         print(f"No table found for URL: {url}")
#         return None
    
#     # Extract the total available budget for the year (last <th> in <thead>)
#     available_budget_tag = table.find('thead').find_all('th')[-1]  # Find the last <th>
#     available_budget = (
#         float(re.sub(r'[^\d.]', '', available_budget_tag.get_text(strip=True))) * 1000
#         if available_budget_tag else None
#     )
    
#     # Extract table headers
#     headers = [th.get_text(strip=True) for th in table.find('thead').find_all('th')]
    
#     # Extract table rows
#     rows = []
#     for tr in table.find('tbody').find_all('tr'):
#         # Extract row cells
#         cells = [td.get_text(strip=True) for td in tr.find_all('td')]
        
#         # Check the class of the <tr> tag for "success" or "discarded"
#         approved = 1 if 'success' in tr.get('class', []) else 0
        
#         # Append cells and approval status
#         rows.append(cells + [approved])
    
#     # Add "Approved" column to the headers
#     headers.append('Approved')
    
#     # Create a DataFrame
#     df = pd.DataFrame(rows, columns=headers)
#     df['Year'] = year  # Add a 'Year' column
#     df['Available Budget'] = available_budget  # Add the total budget for the year to every row
#     return df

# # Scrape and clean tables for all years
# budget_dataframes = [
#     scrape_and_clean_budget_table(url, year) for year, url in budget_urls.items()
# ]

# # Combine all dataframes into one
# budget_jena_df = pd.concat(budget_dataframes, ignore_index=True)

# # Clean and transform the DataFrame
# budget_jena_df['Preis'] = budget_jena_df['Preis'].str.extract(r'(\d+)').astype(float) * 1000
# budget_jena_df['Stimmen'] = budget_jena_df['Stimmen'].str.extract(r'(\d+)').astype(int)

# # Rename columns to English
# budget_jena_df.rename(columns={
#     'Vorschlag Titel': 'Proposal Title',
#     'Stimmen': 'Votes',
#     'Preis': 'Price',
#     'Year': 'Year',
#     'Available Budget': 'Budget for this year',
#     'Approved': 'Approved'
# }, inplace=True)

# # Drop unnecessary columns if any remain
# budget_jena_df = budget_jena_df.loc[:, ~budget_jena_df.columns.str.contains('VerfügbareBudgetmittel', na=False)]



# Comments Scraper

This scraper extracts comments for all projects from the `all_projects_df` DataFrame and organizes them into a structured DataFrame:

1. **`df_comments`**: Contains all comments associated with projects.
   - Columns:
     - `URL`: The URL of the project the comment is associated with.
     - `Project`: The title of the project the comment is associated with.
     - `City`: The city the project belongs to (extracted from the URL).
     - `Text`: The content of the comment.
     - `Username`: The name of the user who posted the comment.
     - `Date`: The date the comment was posted.
     - `Likes`: The number of likes the comment received.
     - `Dislikes`: The number of dislikes the comment received.
     - `Total Votes`: The total votes (likes + dislikes) the comment received.


In [127]:
# Good scraper for comments (748 entries)

import requests
from bs4 import BeautifulSoup
import re

# Updated function to extract comments from a single page
def extract_comments_from_page(soup):
    comments_data = []
    comments_section = soup.find_all('div', class_='comment small-12')
    
    for comment in comments_section:
        # Extract comment text
        comment_text = comment.find('p').get_text(strip=True) if comment.find('p') else None
        
        # Extract username
        username_tag = comment.find('span', class_='user-name')
        username = username_tag.get_text(strip=True) if username_tag else None
        
        # Extract date
        date_tag = comment.find('div', class_='comment-info').find_all('a')[-1]
        date = date_tag.get_text(strip=True) if date_tag else None

        
        # Extract likes and dislikes (clean and convert to integer)
        likes_tag = comment.find('span', class_='in-favor')
        likes = int(re.sub(r'\D', '', likes_tag.get_text(strip=True))) if likes_tag else 0
        
        dislikes_tag = comment.find('span', class_='against')
        dislikes = int(re.sub(r'\D', '', dislikes_tag.get_text(strip=True))) if dislikes_tag else 0
        
        # Extract total votes (clean and convert to integer)
        total_votes = likes + dislikes
        
        comments_data.append({
            'Text': comment_text,
            'Username': username,
            'Date': date,
            'Likes': likes,
            'Dislikes': dislikes,
            'Total Votes': total_votes
        })
    return comments_data


# Function to extract city name from the base URL
def extract_city_name(base_url):
    # Words to remove from the city name
    remove_words = ['mitmachen', 'Mitmachen', 'mitwirken', 'Smarte', 'region', 'unser', 'mitgestalten', 'gestalten', 'machmit', 'dialog', 'consul', 'www', 'de', 'https', 'com']

    # Split the URL into parts (by '.' or '/')
    parts = base_url.replace('https://', '').replace('http://', '').split('.')
    all_parts = [part.split('/')[0] for part in parts]  # Handle cases where "/" exists after domain

    # Remove known unwanted words and empty strings
    filtered_parts = [part for part in all_parts if part.lower() not in remove_words and part]

    # Return the first relevant part (assumes city name is left after filtering)
    city = filtered_parts[0].replace('-', ' ').capitalize() if filtered_parts else "Unknown"

    # Remove unwanted words from city name
    for word in remove_words:
        city = city.replace(word, '')
    

    return city.strip().capitalize()


# Scrape all comments across pages (pagination logic remains the same)
def scrape_all_comments(base_url):
    comments = []
    page = 1
    
    while True:
        paginated_url = f"{base_url}?page={page}" if page > 1 else base_url
        response = requests.get(paginated_url)
        
        if response.status_code != 200:
            print(f"Failed to load page {page} for URL: {base_url}")
            break
        
        soup = BeautifulSoup(response.content, 'html.parser')
        comments_on_page = extract_comments_from_page(soup)
        
        if not comments_on_page:  # Stop if no comments on the page
            break
        
        comments.extend(comments_on_page)
        page += 1

    return comments

# Function to scrape the main content and comments for each URL
def scrape_content_and_comments(urls):
    data = []
    
    for url in urls:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to load URL: {url}")
            continue
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Scrape main content
        title = soup.find('title').get_text(strip=True) if soup.find('title') else None
        content_div = soup.find('div', class_='flex-layout')
        content = content_div.get_text(strip=True) if content_div else None
        
        # Scrape comments
        comments = scrape_all_comments(url)
        
        data.append({
            'URL': url,
            'Title': title,
            'Content': content,
            'Comments': comments
        })
    
    return data

# Scrape comments for all project URLs from all_projects_df
urls = all_projects_df['Project URL'].tolist()  # Use the 'Project URL' column from all_projects_df
scraped_data = scrape_content_and_comments(urls)

# Create structured DataFrame for comments
comments_data = []
for item in scraped_data:
    for comment in item['Comments']:
        comment['URL'] = item['URL']  # Link comment to the project URL
        # Extract city name from URL
        city = extract_city_name(item['URL'])
        comment['City'] = city
        comments_data.append(comment)

# Create the comments DataFrame
df_comments = pd.DataFrame(comments_data)

# Create a mapping from URL to Project Title
url_to_title = all_projects_df.set_index('Project URL')['Project Title'].to_dict()

# Add a 'Project' column to df_comments using the mapping
df_comments['Project'] = df_comments['URL'].map(url_to_title)
df_comments = df_comments[['URL', 'Project', 'City'] + [col for col in df_comments.columns if col not in ['URL', 'Project', 'City']]]

In [130]:
df_comments

Username
Lars Löw           34
klaus.kleiner77    32
Der,wo             31
PM                 18
Klaus.kleiner77    16
                   ..
Julius Kuhn         1
Juliane88           1
Juliane Fuchs       1
Julian Sing         1
🐙                   1
Name: count, Length: 387, dtype: int64

### Additional cleaaning and structuring for Sieburg (review if it's needed) !

In [133]:
# import re

# # Enhanced function to extract all logical parts, including "Unterstützer*innen"
# def extract_full_data_with_supporters(content):
#     # Extract title (everything before the first date)
#     title_match = re.search(r'^(.*?)(\r|\d{1,2}\.\s\w+\s\d{4})', content)
#     title = title_match.group(1).strip() if title_match else None

#     # Extract date
#     date_match = re.search(r'\d{1,2}\.\s\w+\s\d{4}', content)
#     date = date_match.group(0) if date_match else None

#     # Extract comments count
#     comments_match = re.search(r'(\d+)\sKommentare', content)
#     comments = int(comments_match.group(1)) if comments_match else 0

#     # Extract tags (sections with numbers or + signs)
#     tags_match = re.findall(r'(\d{1,2}[-+]\d{1,2}|\d{2}\+)', content)
#     tags = ', '.join(tags_match) if tags_match else None

#     # Extract description (everything after "Geselliges Beisammensein" or similar patterns)
#     description_start = re.search(r'(Geselliges Beisammensein|Angebotslandkarte)', content)
#     description = content[description_start.start():].strip() if description_start else None

#     # Extract username
#     username_match = re.search(r'(\w+\s\w+|Beigetreten am:.*?\d{4})', content)
#     username = username_match.group(1).split('Beigetreten am:')[0].strip() if username_match else None

#     # Extract Vorschläge count
#     vorschlaege_match = re.search(r'Vorschläge(\d+)', content)
#     vorschlaege = int(vorschlaege_match.group(1)) if vorschlaege_match else 0

#     # Extract Konto verification status
#     konto_match = re.search(r'(Konto\s(verifiziert|ist nicht verifiziert))', content)
#     konto_status = konto_match.group(2) if konto_match else None

#     # # Extract registration date
#     # registration_match = re.search(r'Beigetreten am:\s(\d{1,2}\.\s\w+\s\d{4})', content)
#     # registration_date = registration_match.group(1) if registration_match else None

#     # Extract number of Unterstützer*innen
#     supporters_match = re.search(r'(\d+)\sUnterstützer\*in', content)
#     supporters = int(supporters_match.group(1)) if supporters_match else 0

#     return title, date, comments, tags, description, username, vorschlaege, konto_status, supporters

# # Apply the enhanced function to the DataFrame and create new columns
# df_sieburg[['Title', 'Date', 'Comments', 'Tags', 'Description', 'Username', 'Vorschläge', 'Konto Status', 'Supporters']] = df_sieburg['Content'].apply(
#     lambda x: pd.Series(extract_full_data_with_supporters(x))
# )


# # Function to clean description considering keywords, numeric patterns, and refined starting logic
# def clean_description_advanced(content):
#     # Define keywords that mark the beginning of the description
#     keywords = [
#         'Geselliges Beisammensein', 'Natur', 'Hilfe & Beratung', 'Bildung', 
#         'Musik', 'Bewegung', 'Glaube', 'Kulinarisches', 'Kunst & Kultur', 'Sonstiges',
#     ]
    
#     # Check for keywords first
#     for keyword in keywords:
#         if keyword in content:
#             start_idx = content.find(keyword) + len(keyword)
#             description = content[start_idx:].strip()
#             description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
#             return description

#     # If no keyword is found, check for numeric patterns like "18-24, 25-49, etc."
#     numeric_pattern = re.search(r'(\d{1,2}[-+]\d{1,2}|\d{2}\+)', content)
#     if numeric_pattern:
#         start_idx = numeric_pattern.end()
#         description = content[start_idx:].strip()
#         description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
#         return description

#     # As a fallback, find the first capital letter, quote, or digit to mark the start
#     fallback_match = re.search(r'[A-Z"0-9]', content)
#     if fallback_match:
#         start_idx = fallback_match.start()
#         description = content[start_idx:].strip()
#         description = re.split(r'(Kommentare\(.*?\)|registrieren)', description)[0].strip()
#         return description

#     # If nothing works, return the content as is
#     return content

# # Apply the advanced cleaning function to the Description column
# df_sieburg['Description'] = df_sieburg['Content'].apply(clean_description_advanced)
